Este Jupyter Notebook tiene como objetivo principal desarrollar un sistema de recomendación de juegos que permita a los usuarios descubrir nuevos títulos de su interés. Para lograrlo, hemos implementado los dos enfoques diferentes propuestos: uno basado en la relación ítem-ítem y otro en el filtro user-item.

En el primer enfoque, el modelo se basa en la similitud entre juegos. Al ingresar un juego en particular, el sistema generará una lista de 5 juegos recomendados que comparten similitudes con el juego de referencia. Esta metodología es ideal para aquellos usuarios que desean encontrar juegos relacionados o similares a uno que ya les guste.

Por otro lado, en el segundo enfoque, el modelo se centra en el usuario. Al proporcionar el nombre o el identificador de un usuario, el sistema identifica a otros usuarios con gustos y preferencias similares. Luego, se recomiendan juegos que han sido apreciados por esos usuarios afines. Esta metodología se enfoca en personalizar las recomendaciones en función del perfil de cada usuario.

En conjunto, estos dos enfoques ofrecen a los usuarios la posibilidad de explorar juegos de manera eficiente y personalizada, ya sea en función de sus juegos favoritos o de su perfil de usuario. 

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Inicialización

Importamos las librerías que nos servirán para poder procesar, visualizar y explorar nuestros datos de manera efectiva, la librería `pandas`, `scipy`, `sklearn`.

In [27]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import pyarrow as pa
import pyarrow.parquet as pq

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extracción de los datos

Como se expone en el Análisis Exploratorio de Datos (EDA), nuestro modelo de recomendación de juegos se basa en la asignación de puntajes del 1 al 5 a cada juego, considerando análisis de sentimiento y recomendaciones de usuarios. Esto permite proporcionar recomendaciones personalizadas. La función central en este proceso asegura que los usuarios reciban recomendaciones precisas basadas en la calidad percibida de un juego y las preferencias de la comunidad.

Para realizar los análisis requeridos, extraeremos los datos esenciales del conjunto extraido del **EDA** `recommendation_model`. Estos datos nos permitirán realizar los análisis requeridos.

In [5]:
recommendation_model = pd.read_csv('recommendation_model.csv')

In [6]:
recommendation_model.head(3)

,user_id,item_id,recommendation,item_name
0,76561197970982479,1250,5,Killing Floor
1,EndAtHallow,1250,1,Killing Floor
2,76561198107847795,1250,5,Killing Floor


Creamos una copia del DataFrame original, manteniendo solo las columnas relevantes para nuestro análisis, que son 'user_id', 'item_name' y 'recommendation'. Luego, pivotamos el DataFrame para obtener una matriz de calificaciones con 'user_id' como índices, los juegos ('item_name') como columnas y las calificaciones ('recommendation') como valores. Esta matriz será la base sobre la cual realizaremos la normalización de las calificaciones.

In [7]:
# Crea una copia del DataFrame original con las columnas relevantes
df_ratings = recommendation_model[['user_id', 'item_name', 'recommendation']]

# Pivota el DataFrame para obtener una matriz de calificaciones
pivot_ratings = df_ratings.pivot_table(index=['user_id'], columns=['item_name'], values='recommendation')


In [8]:
pivot_ratings

item_name,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,123 Slaughter Me Street,140,16 Bit Arena,200% Mixed Juice!,...,ibb & obb,inMomentum,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ace--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ionex--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2SV-vuLB-Kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Beave-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zv_odd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zvanik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zynxgameth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Normalización del DataFrame; tiene como objetivo transformar las calificaciones de los usuarios de manera que estén centradas en cero y escaladas en función de su variabilidad. Esta transformación es esencial para garantizar que las calificaciones sean comparables y coherentes

In [9]:
# Normalización del dataframe 'piv'
#pivot_ratings_normalized = pivot_ratings.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)


In [37]:
# Elimina filas y columnas con valores NaN
pivot_ratings = pivot_ratings.dropna(axis=0, how='all').dropna(axis=1, how='all')

# Calcula la media de calificaciones por usuario
user_means = pivot_ratings.mean(axis=1)

# Resta la media a las calificaciones (centra las calificaciones en cero)
pivot_ratings_centered = pivot_ratings.sub(user_means, axis=0)

# Calcula la diferencia entre el valor máximo y mínimo de calificaciones por usuario
rating_range = pivot_ratings_centered.max(axis=1) - pivot_ratings_centered.min(axis=1)

# Divide las calificaciones centradas por la diferencia de rango (normalización)
pivot_ratings_normalized = pivot_ratings_centered.div(rating_range, axis=0)


Abordamos la presencia de valores nulos en nuestro conjunto de datos, reemplazándolos de manera adecuada para asegurarnos de que todos los datos estén completos.Luego ajustamos la estructura de los datos para facilitar el análisis:
-Transposición de la matriz para que los juegos se conviertan en las filas y los usuarios en las columnas, lo que mejora la interpretación de los resultados.
-Eliminacion de las columnas que consisten en ceros en su totalidad, lo que nos permite centrarnos en las interacciones significativas entre usuarios y juegos.

In [38]:
# Rellena los valores nulos con 0
pivot_ratings_normalized.fillna(0, inplace=True)

# Transponer la matriz para que los juegos sean las filas y los usuarios las columnas
pivot_ratings_normalized = pivot_ratings_normalized.T

# Eliminar columnas que consisten en ceros en su totalidad
pivot_ratings_normalized = pivot_ratings_normalized.loc[:, (pivot_ratings_normalized != 0).any(axis=0)]



In [39]:
pivot_ratings_normalized

user_id,-2SV-vuLB-Kg,-GM-Dragon,-PRoSlayeR-,-SEVEN-,-_PussyDestroyer_-,0-3-0,00000000000000000001227,00True,01189958889189157253,01221733,...,zile98,zixwot,zombi_anon,zomgieee,zoozles,zrustz16,zuilde,zuzuga2003,zv_odd,zyr0n1c
item_name,,,,,,,,,,,,,,,,,,,,,
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
12 Labours of Hercules,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
resident evil 4 / biohazard 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0


Transformamos la matriz de calificaciones normalizadas en una 'matriz dispersa'.

In [40]:
# Convierte la matriz normalizada en una matriz dispersa
sparse_ratings = sp.sparse.csr_matrix(pivot_ratings_normalized.values)

In [41]:
sparse_ratings

<2639x6188 sparse matrix of type '<class 'numpy.float64'>'
	with 22954 stored elements in Compressed Sparse Row format>

la matriz tiene dimensiones 2207x6188, donde 6188 representa el número de usuarios y 2207 el número de juegos. Podemos observar que la  mayoría de los valores son ceros, lo que es típico en sistemas de recomendación donde no todos los usuarios califican todos los juegos. Los valores almacenados son números de punto flotante ('float64') y, a pesar de ser una matriz dispersa, contiene 22954 elementos distintos de cero, que corresponden a las calificaciones proporcionadas por los usuarios para los juegos.

Calculamos la similitud del coseno entre los juegos, una medida esencial para identificar la similitud entre elementos en un espacio multidimensional. Esto se logra utilizando la función 'cosine_similarity' en nuestros datos previamente transformados en una matriz dispersa. Esta medida de similitud del coseno nos permitirá identificar qué juegos se asemejan más entre sí, lo que es fundamental para generar recomendaciones precisas en nuestro sistema.

In [42]:
# Calcula la similitud del coseno entre juegos
similitudes_coseno = cosine_similarity(sparse_ratings)


Creamos un 'DataFrame de similitud de juegos' (df_item_similarity). 

In [43]:
#item similarity dataframe
df_item_similarity = pd.DataFrame(similitudes_coseno, index = pivot_ratings_normalized.index, columns = pivot_ratings_normalized.index)

Para optimizar el uso de memoria y mejorar la eficiencia en el manejo de conjuntos de datos extensos, se transforman los datos de la matriz normalizada en un formato de matriz dispersa (sparse matrix). En una matriz dispersa, solo se almacenan los valores distintos de cero, junto con su ubicación en la matriz, en lugar de ocupar espacio para todos los valores, incluso aquellos que son ceros. Esta técnica permite un uso más eficiente de los recursos y es especialmente beneficiosa cuando la mayoría de los valores son nulos.

# Carga del Conjunto de Datos

In [28]:
# Guardar en Parquet
table = pa.Table.from_pandas(df_item_similarity)
pq.write_table(table, 'df_item_similarity.parquet')
print(f'Se ha guardado el archivo df_item_similarity.parquet en la misma carpeta.')

Se ha guardado el archivo df_item_similarity.parquet en la misma carpeta.


# Desarrollo de la Funciones requeridas

## recomendacion_juego

En la siguiente función nos permitirá ofrecer recomendaciones de juegos similares. Aplicando la métrica de similitud del coseno para lograr este objetivo. La función 'recomendacion_juego(id_de_producto)' se ha desarrollado específicamente para brindar a los usuarios una lista de 5 juegos recomendados que son similares al juego cuyo ID se ingresa como parámetro.

La similitud del coseno es una técnica poderosa que nos permite comparar la similitud entre los juegos en función de las calificaciones de los usuarios. Al considerar las preferencias de los usuarios y la similitud en las calificaciones, podemos generar recomendaciones precisas y relevantes para enriquecer la experiencia de nuestros usuarios.

A continuación, se presenta la función 'recomendacion_juego' que hemos implementado y que se basa en la similitud del coseno. Al ingresar el ID de un juego, esta función calculará la similitud de ese juego con otros juegos en nuestro conjunto de datos y proporcionará una lista de los 5 juegos más similares como recomendación. Esto permitirá a nuestros usuarios descubrir nuevos juegos que se ajusten a sus gustos y preferencias.

In [44]:
def game_recommendation(id_juego, num_recomendaciones=5):
    if id_juego in df_item_similarity:
        # Selecciona la fila correspondiente al juego ingresado
        juego_similaridades = df_item_similarity[id_juego]
        
        # Ordena las similitudes en orden descendente
        juegos_similares = juego_similaridades.sort_values(ascending=False)
        
        # Excluye el juego ingresado (similitud con sí mismo será 1) y obtiene solo los nombres de los juegos
        juegos_similares = juegos_similares[1:num_recomendaciones + 1]
        
        print(f"Juegos similares a '{id_juego}':")
        for i, (juego, similitud) in enumerate(juegos_similares.items(), 1):
            print(f"Recomendación {i}: {juego}")
    else:
        print("El juego ingresado no está en la base de datos.")

In [48]:
game_recommendation('Revenge of the Titans')

Juegos similares a 'Revenge of the Titans':
Recomendación 1: Droplitz
Recomendación 2: Gratuitous Space Battles
Recomendación 3: Demigod
Recomendación 4: Realm of the Mad God
Recomendación 5: Garry's Mod
